In [1]:
import tensorflow as tf
import numpy as np
import json

In [5]:
###
### Citest json files si creeaza lista de cuvinte unice din venues. Scrie rezultatul in fisier.
### return numarul de cuvinte.
###
def creeaza_lista_cuvinte_unice():
    lista_unica_cuvinte = []
    
    ## Citeste lista de venues
    r = open('../Date/Initiale/lista_venue_completa.txt', 'r')
    lista_venues = json.load(r)
    r.close()
    
    ## Creeaza lista de cuvinte unice din venues
    for venue in lista_venues:
        lista_unica_cuvinte += venue
    lista_unica_cuvinte = set(lista_unica_cuvinte)
    
    w = open('../DATE/lista_unica_cuvinte.txt','w')
    w.write(json.dumps(list(lista_unica_cuvinte)))
    w.close()

    return len(lista_unica_cuvinte)

###
### Citeste lista de cuvinte unice si creeaza dictionar {cuvant:id}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_word2int(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    word2int = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        word2int[cuvant] = i
        
    w = open(out_file,'w')
    w.write(json.dumps(word2int))
    w.close()
    
    
###
### Citeste lista de cuvinte unice si creeaza dictionar {id:cuvant}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_int2word(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    int2word = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        int2word[i] = cuvant
        
    w = open(out_file,'w')
    w.write(json.dumps(int2word))
    w.close()

###
### Creeaza un vector de tip one hot.
### param index: pozitia din vector pe care se va pune 1.
### param len_vect: lungimea vectorului.
### return hot_vect: vectorul rezultat.
###
def one_hot(index, len_vect):
    hot_vect = np.zeros(len_vect)
    hot_vect[index] = 1
    return hot_vect

###
###
###
def creeaza_lista_vecini(word2int_file, docs_file, DIM_FEREASTRA, DIM_VOCABULAR):
    # Citeste word2int
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    # Citeste lista documente
    r = open(docs_file, 'r')
    lista_doc = json.load(r)
    r.close()
    
    X_train = [] 
    Y_train = [] 
    
    for doc in lista_doc:
        for word_index, word in enumerate(doc):
            for vecin in doc[max(word_index - DIM_FEREASTRA, 0) : min(word_index + DIM_FEREASTRA, len(doc)) + 1] : 
                if vecin != word:
                    X_train.append(one_hot(word2int[ word ], DIM_VOCABULAR))
                    Y_train.append(one_hot(word2int[ vecin ], DIM_VOCABULAR))

    X_train = np.asarray(X_train)
    Y_train = np.asarray(Y_train)
    
    return X_train, Y_train

def log(word2int_file, vectors, file):
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    dict_ = {}
    for elem in word2int:
        dict_[elem] = vectors[word2int[elem]].tolist()
    w = open (file, 'w')
    w.write(json.dumps(dict_))
    w.close()       
    

def train ():
    DIM_VOCABULAR = creeaza_lista_cuvinte_unice()
    #creeaza_word2int('../DATE/lista_unica_cuvinte.txt','../DATE/word2int.txt')
    #creeaza_int2word('../DATE/lista_unica_cuvinte.txt','../DATE/int2word.txt')
    
    DIM_FEREASTRA = 2
    X_train, Y_train = creeaza_lista_vecini('../Date/Initiale/word2int_venues.txt', '../Date/Initiale/lista_venue_completa.txt', DIM_FEREASTRA, DIM_VOCABULAR)
    
    #### Creare retea
    X = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))
    Y_eticheta = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))

    ## Din layer de input in hidden layer
    EMBEDDING_DIM = 50
    W1 = tf.Variable(tf.random_normal([DIM_VOCABULAR, EMBEDDING_DIM]))
    b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
    layer_hidden = tf.add(tf.matmul(X,W1), b1)

    ## Din hidden layer in output layer
    W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, DIM_VOCABULAR]))
    b2 = tf.Variable(tf.random_normal([DIM_VOCABULAR]))
    output_layer = tf.nn.softmax(tf.add( tf.matmul(layer_hidden, W2), b2))
    
    #### Instantiere model tensorflow
    with tf.device('/gpu:0'):
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)

        # Functia de loss -> cross entropy
        cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y_eticheta * tf.log(tf.clip_by_value(output_layer,1e-10,1.0)), reduction_indices=[1]))
        #is actually a horrible way of computing the cross-entropy. In some samples, certain classes could be excluded with certainty after a while, resulting in y_conv=0 for that sample. That's normally not a problem since you're not interested in those, but in the way cross_entropy is written there, it yields 0*log(0) for that particular sample/class. Hence the NaN.
        
        # compute_gradients + apply_gradients = minimize
        learning_rate = 0.02
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)

        i = 0
        loss_stochastic = []
        while(1):
        
            for k in range(DIM_VOCABULAR):
                x_rand = X_train[k].reshape(1,1416) # Transpose to the correct shape
                y_rand = Y_train[k].reshape(1,1416)
                
                sess.run(train_step, feed_dict={X: x_rand, Y_eticheta:y_rand})
                
                temp_loss = sess.run(cross_entropy_loss, feed_dict={X: x_rand, Y_eticheta:y_rand})
                loss_stochastic.append(temp_loss)

                if k % 100 == 0:
                    print('Loss = ', temp_loss, i)
                    w = open('nf/loss.txt','a')
                    w.write(str(temp_loss))
                    w.write(' ')
                    w.close()
                
        
        return vectors

In [7]:
vectors = train()

Loss =  23.02585 0
Loss =  13.849758 0
Loss =  23.02585 0
Loss =  10.537686 0
Loss =  13.896045 0
Loss =  1.294995 0
Loss =  23.02585 0
Loss =  7.1327314 0
Loss =  2.0443997 0
Loss =  23.02585 0
Loss =  14.740279 0
Loss =  0.0546788 0
Loss =  23.02585 0
Loss =  1.6722246 0
Loss =  0.06656637 0
Loss =  23.02585 0
Loss =  4.5972223 0
Loss =  23.02585 0
Loss =  0.14506325 0
Loss =  4.7608743 0
Loss =  1.7701385 0
Loss =  12.608432 0
Loss =  1.9402417 0
Loss =  0.99710244 0
Loss =  23.02585 0
Loss =  14.454337 0
Loss =  0.022078343 0
Loss =  23.02585 0
Loss =  0.10127109 0
Loss =  0.02688481 0
Loss =  23.02585 0
Loss =  3.7923095 0
Loss =  18.153769 0
Loss =  0.22336328 0
Loss =  3.5557823 0
Loss =  0.81413364 0
Loss =  6.588174 0
Loss =  0.17046851 0
Loss =  0.34945184 0
Loss =  16.739159 0
Loss =  11.74733 0
Loss =  0.0429709 0
Loss =  19.56345 0
Loss =  0.12976949 0
Loss =  0.032373827 0
Loss =  23.02585 0
Loss =  2.194183 0
Loss =  13.15421 0
Loss =  0.3214925 0
Loss =  2.977007 0
Loss

Loss =  0.81745154 0
Loss =  0.35087305 0
Loss =  2.2859054 0
Loss =  0.44239026 0
Loss =  0.97753465 0
Loss =  2.0670161 0
Loss =  0.19073647 0
Loss =  0.25527543 0
Loss =  0.39546493 0
Loss =  0.2584127 0
Loss =  0.505441 0
Loss =  6.4540277 0
Loss =  0.24338163 0
Loss =  0.45734566 0
Loss =  23.02585 0
Loss =  0.8334912 0
Loss =  0.34820855 0
Loss =  2.275361 0
Loss =  0.43426424 0
Loss =  0.9853001 0
Loss =  2.082921 0
Loss =  0.20480774 0
Loss =  0.24231894 0
Loss =  0.39848626 0
Loss =  0.26472494 0
Loss =  0.49526647 0
Loss =  6.247865 0
Loss =  0.24794587 0
Loss =  0.45890573 0
Loss =  23.02585 0
Loss =  0.83214647 0
Loss =  0.3436091 0
Loss =  2.2701902 0
Loss =  0.42473006 0
Loss =  0.990042 0
Loss =  2.1539688 0
Loss =  0.22533178 0
Loss =  0.22351219 0
Loss =  0.39982405 0
Loss =  0.27440485 0
Loss =  0.48760927 0
Loss =  6.0136814 0
Loss =  0.24938263 0
Loss =  0.45939195 0
Loss =  23.02585 0
Loss =  0.82873785 0
Loss =  0.33918062 0
Loss =  2.275725 0
Loss =  0.41599715 0

Loss =  0.49424714 0
Loss =  23.02585 0
Loss =  0.8199141 0
Loss =  0.34318057 0
Loss =  2.3909843 0
Loss =  0.32658684 0
Loss =  0.9569321 0
Loss =  1.9853796 0
Loss =  0.16173033 0
Loss =  0.29128966 0
Loss =  0.4080445 0
Loss =  0.31246045 0
Loss =  0.52453643 0
Loss =  2.9629414 0
Loss =  0.36075607 0
Loss =  0.66823643 0
Loss =  23.02585 0
Loss =  0.82304764 0
Loss =  0.35114264 0
Loss =  2.40038 0
Loss =  0.32994536 0
Loss =  0.95790845 0
Loss =  1.97077 0
Loss =  0.17042695 0
Loss =  0.29166347 0
Loss =  0.4130892 0
Loss =  0.3125973 0
Loss =  0.52143455 0
Loss =  2.946645 0
Loss =  0.35585573 0
Loss =  0.63591194 0
Loss =  23.02585 0
Loss =  0.8244477 0
Loss =  0.35337967 0
Loss =  2.4063559 0
Loss =  0.3309694 0
Loss =  0.9561547 0
Loss =  1.9567548 0
Loss =  0.17405431 0
Loss =  0.29292542 0
Loss =  0.41514042 0
Loss =  0.3134947 0
Loss =  0.5208374 0
Loss =  2.9340265 0
Loss =  0.357852 0
Loss =  0.6179721 0
Loss =  23.02585 0
Loss =  0.82640576 0
Loss =  0.35562128 0
Loss =

Loss =  0.45152563 0
Loss =  0.54490674 0
Loss =  23.02585 0
Loss =  0.87200063 0
Loss =  0.39514816 0
Loss =  2.382187 0
Loss =  0.35255483 0
Loss =  0.978964 0
Loss =  1.8096123 0
Loss =  0.1866345 0
Loss =  0.3507889 0
Loss =  0.4123823 0
Loss =  0.38109756 0
Loss =  0.49066234 0
Loss =  2.7711995 0
Loss =  0.45381516 0
Loss =  0.545038 0
Loss =  23.02585 0
Loss =  0.8706986 0
Loss =  0.3962866 0
Loss =  2.3836374 0
Loss =  0.35310805 0
Loss =  0.98307306 0
Loss =  1.8048285 0
Loss =  0.18817466 0
Loss =  0.35149598 0
Loss =  0.4278344 0
Loss =  0.38222483 0
Loss =  0.4896024 0
Loss =  2.7785528 0
Loss =  0.4554184 0
Loss =  0.54501224 0
Loss =  23.02585 0
Loss =  0.8705464 0
Loss =  0.39751408 0
Loss =  2.3852758 0
Loss =  0.35370815 0
Loss =  0.9870038 0
Loss =  1.8019211 0
Loss =  0.19060224 0
Loss =  0.35311753 0
Loss =  0.44298 0
Loss =  0.38446838 0
Loss =  0.4881354 0
Loss =  2.7847514 0
Loss =  0.46731263 0
Loss =  0.5417615 0
Loss =  23.02585 0
Loss =  0.8700436 0
Loss =  0

Loss =  0.5248489 0
Loss =  0.52668786 0
Loss =  23.02585 0
Loss =  0.8605232 0
Loss =  0.4511938 0
Loss =  2.3466253 0
Loss =  0.37536705 0
Loss =  1.0450693 0
Loss =  2.2994087 0
Loss =  0.19195072 0
Loss =  0.3990644 0
Loss =  0.52873164 0
Loss =  0.43941677 0
Loss =  0.4920404 0
Loss =  2.7637997 0
Loss =  0.52761954 0
Loss =  0.5257541 0
Loss =  23.02585 0
Loss =  0.8616952 0
Loss =  0.45278704 0
Loss =  2.3332164 0
Loss =  0.3775312 0
Loss =  1.0498002 0
Loss =  2.2241492 0
Loss =  0.19075882 0
Loss =  0.40132368 0
Loss =  0.5306412 0
Loss =  0.44159108 0
Loss =  0.4876011 0
Loss =  2.762991 0
Loss =  0.52994704 0
Loss =  0.5250485 0
Loss =  23.02585 0
Loss =  0.86423117 0
Loss =  0.45389387 0
Loss =  2.3230171 0
Loss =  0.38057277 0
Loss =  1.0541282 0
Loss =  2.151781 0
Loss =  0.19162132 0
Loss =  0.40337497 0
Loss =  0.5320101 0
Loss =  0.44289747 0
Loss =  0.4862051 0
Loss =  2.7636948 0
Loss =  0.53223604 0
Loss =  0.5246302 0
Loss =  23.02585 0
Loss =  0.8683593 0
Loss =  

Loss =  0.60039055 0
Loss =  0.50767064 0
Loss =  23.02585 0
Loss =  0.9199793 0
Loss =  0.5273547 0
Loss =  2.159396 0
Loss =  0.4287999 0
Loss =  1.0809288 0
Loss =  2.1488008 0
Loss =  0.20044766 0
Loss =  0.39946532 0
Loss =  0.5231894 0
Loss =  0.46121174 0
Loss =  0.5582022 0
Loss =  2.7718763 0
Loss =  0.60159683 0
Loss =  0.50710505 0
Loss =  23.02585 0
Loss =  0.9246781 0
Loss =  0.5312687 0
Loss =  2.1586637 0
Loss =  0.43032804 0
Loss =  1.0827768 0
Loss =  2.1604147 0
Loss =  0.19943345 0
Loss =  0.3994152 0
Loss =  0.5225382 0
Loss =  0.46199054 0
Loss =  0.5599872 0
Loss =  2.7703402 0
Loss =  0.60271573 0
Loss =  0.5065923 0
Loss =  23.02585 0
Loss =  0.9295152 0
Loss =  0.53565747 0
Loss =  2.1580675 0
Loss =  0.43193948 0
Loss =  1.0845511 0
Loss =  2.1715655 0
Loss =  0.19833386 0
Loss =  0.39935565 0
Loss =  0.5218434 0
Loss =  0.46290505 0
Loss =  0.56166226 0
Loss =  2.7689636 0
Loss =  0.60378 0
Loss =  0.50612175 0
Loss =  23.02585 0
Loss =  0.9345447 0
Loss =  0

Loss =  0.49862346 0
Loss =  23.02585 0
Loss =  1.1132787 0
Loss =  0.67724025 0
Loss =  2.1566937 0
Loss =  0.4665531 0
Loss =  1.117303 0
Loss =  2.4039917 0
Loss =  0.17156936 0
Loss =  0.39517444 0
Loss =  0.5156517 0
Loss =  0.4865252 0
Loss =  0.57156885 0
Loss =  2.7838876 0
Loss =  0.61399937 0
Loss =  0.4985364 0
Loss =  23.02585 0
Loss =  1.1233804 0
Loss =  0.6823738 0
Loss =  2.1569018 0
Loss =  0.46790242 0
Loss =  1.1178509 0
Loss =  2.411952 0
Loss =  0.17176723 0
Loss =  0.39459765 0
Loss =  0.5152426 0
Loss =  0.48961037 0
Loss =  0.5713224 0
Loss =  2.7838156 0
Loss =  0.6138706 0
Loss =  0.4984953 0
Loss =  23.02585 0
Loss =  1.1332397 0
Loss =  0.6876207 0
Loss =  2.1573815 0
Loss =  0.4693097 0
Loss =  1.118433 0
Loss =  2.5027692 0
Loss =  0.17518297 0
Loss =  0.39733925 0
Loss =  0.51843345 0
Loss =  0.49851406 0
Loss =  0.57244855 0
Loss =  2.7897863 0
Loss =  0.61304724 0
Loss =  0.50240344 0
Loss =  23.02585 0
Loss =  1.1515561 0
Loss =  0.6982016 0
Loss =  2.

KeyboardInterrupt: 